In [0]:
# MAGIC ## 1. Imports e Configuração

# COMMAND ----------

import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
from datetime import datetime

spark = SparkSession.builder \
    .appName("SilverBalancaComercial") \
    .getOrCreate()

In [0]:
# MAGIC ## 2. Configuração de Storage (ABFSS - ADLS Gen2)

# COMMAND ----------

# ✅ Recuperar Storage Account
TGT_STORAGE_ACCOUNT = dbutils.secrets.get(scope="acelera-grupo-5-kv", key="tgt-storage-account")

# ✅ Containers
SILVER_CONTAINER = "silver"

# ✅ USAR ABFSS:// (Azure Data Lake Storage Gen2) - MESMO PROTOCOLO DO BRONZE
SILVER_BASE_PATH = f"abfss://{SILVER_CONTAINER}@{TGT_STORAGE_ACCOUNT}.dfs.core.windows.net/balancacomercial"
CONTROL_TABLE_PATH = f"abfss://{SILVER_CONTAINER}@{TGT_STORAGE_ACCOUNT}.dfs.core.windows.net/metadata/silver_control_balanca"

print(f"✅ Storage Account: {TGT_STORAGE_ACCOUNT}")
print(f"✅ Silver Container: {SILVER_CONTAINER}")
print(f"✅ Protocolo: abfss:// (ADLS Gen2)")
print(f"📂 Silver Base Path: {SILVER_BASE_PATH}")
print(f"📂 Control Table Path: {CONTROL_TABLE_PATH}")
print(f"🗄️ Bronze: Leitura via Hive Metastore (bronze_balancacomercial)")

# COMMAND ----------

# 🧪 VALIDAR ACESSO ao container Silver
print("\n🧪 Validando acesso ao container silver...")

try:
    test_path = f"abfss://{SILVER_CONTAINER}@{TGT_STORAGE_ACCOUNT}.dfs.core.windows.net/"
    files = dbutils.fs.ls(test_path)
    print(f"✅ Container 'silver' acessível - {len(files)} itens encontrados")
    
except Exception as e:
    print(f"❌ ERRO: Cluster não tem acesso ao container 'silver'")
    print(f"\n⚠️ Verifique:")
    print(f"   1. Container 'silver' existe no Storage Account")
    print(f"   2. Service Principal/Managed Identity tem permissões")
    print(f"   3. Role necessária: 'Storage Blob Data Contributor'")
    print(f"\nErro: {str(e)[:300]}")
    raise

In [0]:
# MAGIC ## 3. Otimizações Spark

# COMMAND ----------

spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "true")

print("✅ Otimizações Spark configuradas")

In [0]:
# MAGIC ## 4. Criação de Databases

# COMMAND ----------

spark.sql("CREATE DATABASE IF NOT EXISTS silver_balancacomercial")
spark.sql("CREATE DATABASE IF NOT EXISTS metadata")

print("✅ Databases criados no Hive Metastore")

In [0]:
# MAGIC ## 5. Criação da Tabela de Controle

# COMMAND ----------

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS metadata.silver_control_balanca (
        table_name STRING COMMENT 'Nome da tabela Silver',
        last_ingestion_timestamp TIMESTAMP COMMENT 'Último timestamp processado',
        last_update TIMESTAMP COMMENT 'Timestamp da última atualização',
        records_processed BIGINT COMMENT 'Total de registros processados',
        execution_id STRING COMMENT 'ID da execução'
    )
    USING DELTA
    LOCATION '{CONTROL_TABLE_PATH}'
    COMMENT 'Controle incremental - Silver Balança Comercial'
""")

print("✅ Tabela de controle criada")

# Verificar location
location = spark.sql("DESCRIBE DETAIL metadata.silver_control_balanca").select("location").first()[0]
print(f"📂 Location: {location}")

In [0]:
# MAGIC ## 6. Configuração de Transformações

# COMMAND ----------

table_config = [
    # ========================================
    # DIMENSÕES SIMPLES
    # ========================================
    
    {
        "bronze_table": "pais",
        "silver_table": "silver_pais",
        "key_columns": ["co_pais"],
        "column_mapping": {
            "CO_PAIS": {"rename": "co_pais", "cast": "string", "trim": True},
            "CO_PAIS_ISON3": {"rename": "co_pais_ison3", "cast": "string", "trim": True},
            "CO_PAIS_ISOA3": {"rename": "co_pais_isoa3", "cast": "string", "trim": True},
            "NO_PAIS": {"rename": "no_pais", "cast": "string", "trim": True},
            "NO_PAIS_ING": {"rename": "no_pais_ing", "cast": "string", "trim": True},
            "NO_PAIS_ESP": {"rename": "no_pais_esp", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "Dimensão de países"
    },
    
    {
        "bronze_table": "via",
        "silver_table": "silver_via",
        "key_columns": ["co_via"],
        "column_mapping": {
            "CO_VIA": {"rename": "co_via", "cast": "string", "trim": True},
            "NO_VIA": {"rename": "no_via", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "Dimensão de vias de transporte"
    },
    
    {
        "bronze_table": "urf",
        "silver_table": "silver_urf",
        "key_columns": ["co_urf"],
        "column_mapping": {
            "CO_URF": {"rename": "co_urf", "cast": "string", "trim": True},
            "NO_URF": {"rename": "no_urf", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "Dimensão URF"
    },
    
    {
        "bronze_table": "ncm_unidade",
        "silver_table": "silver_ncm_unidade",
        "key_columns": ["co_unid"],
        "column_mapping": {
            "CO_UNID": {"rename": "co_unid", "cast": "string", "trim": True},
            "NO_UNID": {"rename": "no_unid", "cast": "string", "trim": True},
            "SG_UNID": {"rename": "sg_unid", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "Dimensão de unidades"
    },
    
    {
        "bronze_table": "uf",
        "silver_table": "silver_uf",
        "key_columns": ["sg_uf"],
        "column_mapping": {
            "CO_UF": {"rename": "co_uf", "cast": "string", "trim": True},
            "SG_UF": {"rename": "sg_uf", "cast": "string", "trim": True},
            "NO_UF": {"rename": "no_uf", "cast": "string", "trim": True},
            "NO_REGIAO": {"rename": "no_regiao", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "Dimensão de UF"
    },
    
    {
        "bronze_table": "uf_mun",
        "silver_table": "silver_uf_mun",
        "key_columns": ["co_mun_geo"],
        "column_mapping": {
            "CO_MUN_GEO": {"rename": "co_mun_geo", "cast": "string", "trim": True},
            "NO_MUN": {"rename": "no_mun", "cast": "string", "trim": True},
            "NO_MUN_MIN": {"rename": "no_mun_min", "cast": "string", "trim": True},
            "SG_UF": {"rename": "sg_uf", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "Dimensão de municípios"
    },
    
    {
        "bronze_table": "isic_cuci",
        "silver_table": "silver_isic_cuci",
        "key_columns": ["co_isic_secao", "co_cuci_grupo"],
        "column_mapping": {
            "CO_ISIC_SECAO": {"rename": "co_isic_secao", "cast": "string", "trim": True},
            "NO_ISIC_SECAO": {"rename": "no_isic_secao", "cast": "string", "trim": True},
            "CO_CUCI_GRUPO": {"rename": "co_cuci_grupo", "cast": "string", "trim": True},
            "NO_CUCI_GRUPO": {"rename": "no_cuci_grupo", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "ISIC/CUCI"
    },
    
    {
        "bronze_table": "nbm",
        "silver_table": "silver_nbm",
        "key_columns": ["co_nbm"],
        "column_mapping": {
            "CO_NBM": {"rename": "co_nbm", "cast": "string", "trim": True},
            "NO_NBM": {"rename": "no_nbm", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "NBM"
    },
    
    {
        "bronze_table": "ncm_ppe",
        "silver_table": "silver_ncm_ppe",
        "key_columns": ["co_ppe"],
        "column_mapping": {
            "CO_PPE": {"rename": "co_ppe", "cast": "string", "trim": True},
            "NO_PPE": {"rename": "no_ppe", "cast": "string", "trim": True},
            "NO_PPE_MIN": {"rename": "no_ppe_min", "cast": "string", "trim": True},
            "NO_PPE_ING": {"rename": "no_ppe_ing", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "PPE"
    },
    
    {
        "bronze_table": "ncm_ppi",
        "silver_table": "silver_ncm_ppi",
        "key_columns": ["co_ppi"],
        "column_mapping": {
            "CO_PPI": {"rename": "co_ppi", "cast": "string", "trim": True},
            "NO_PPI": {"rename": "no_ppi", "cast": "string", "trim": True},
            "NO_PPI_MIN": {"rename": "no_ppi_min", "cast": "string", "trim": True},
            "NO_PPI_ING": {"rename": "no_ppi_ing", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "PPI"
    },
    
    {
        "bronze_table": "ncm_fat_agreg",
        "silver_table": "silver_ncm_fat_agreg",
        "key_columns": ["co_fat_agreg"],
        "column_mapping": {
            "CO_FAT_AGREG": {"rename": "co_fat_agreg", "cast": "string", "trim": True},
            "NO_FAT_AGREG": {"rename": "no_fat_agreg", "cast": "string", "trim": True},
            "NO_FAT_AGREG_GP": {"rename": "no_fat_agreg_gp", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "Fator agregação"
    },
    
    {
        "bronze_table": "nbm_ncm",
        "silver_table": "silver_nbm_ncm",
        "key_columns": ["co_nbm"],
        "column_mapping": {
            "CO_NBM": {"rename": "co_nbm", "cast": "string", "trim": True},
            "CO_NCM": {"rename": "co_ncm", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "NBM-NCM"
    },
    
    {
        "bronze_table": "ncm_sh",
        "silver_table": "silver_ncm_sh",
        "key_columns": ["co_sh6"],
        "column_mapping": {
            "CO_SH6": {"rename": "co_sh6", "cast": "string", "trim": True},
            "NO_SH6_POR": {"rename": "no_sh6_por", "cast": "string", "trim": True},
            "NO_SH6_ESP": {"rename": "no_sh6_esp", "cast": "string", "trim": True},
            "NO_SH6_ING": {"rename": "no_sh6_ing", "cast": "string", "trim": True},
            "CO_SH4": {"rename": "co_sh4", "cast": "string", "trim": True},
            "NO_SH4_POR": {"rename": "no_sh4_por", "cast": "string", "trim": True},
            "NO_SH4_ESP": {"rename": "no_sh4_esp", "cast": "string", "trim": True},
            "NO_SH4_ING": {"rename": "no_sh4_ing", "cast": "string", "trim": True},
            "CO_SH2": {"rename": "co_sh2", "cast": "string", "trim": True},
            "NO_SH2_POR": {"rename": "no_sh2_por", "cast": "string", "trim": True},
            "NO_SH2_ESP": {"rename": "no_sh2_esp", "cast": "string", "trim": True},
            "NO_SH2_ING": {"rename": "no_sh2_ing", "cast": "string", "trim": True},
            "CO_NCM_SECROM": {"rename": "co_ncm_secrom", "cast": "string", "trim": True},
            "NO_SEC_POR": {"rename": "no_sec_por", "cast": "string", "trim": True},
            "NO_SEC_ESP": {"rename": "no_sec_esp", "cast": "string", "trim": True},
            "NO_SEC_ING": {"rename": "no_sec_ing", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "Sistema Harmonizado"
    },
    
    {
        "bronze_table": "ncm_cgce",
        "silver_table": "silver_ncm_cgce",
        "key_columns": ["co_cgce_n3"],
        "column_mapping": {
            "CO_CGCE_N3": {"rename": "co_cgce_n3", "cast": "string", "trim": True},
            "NO_CGCE_N3": {"rename": "no_cgce_n3", "cast": "string", "trim": True},
            "NO_CGCE_N3_ING": {"rename": "no_cgce_n3_ing", "cast": "string", "trim": True},
            "NO_CGCE_N3_ESP": {"rename": "no_cgce_n3_esp", "cast": "string", "trim": True},
            "CO_CGCE_N2": {"rename": "co_cgce_n2", "cast": "string", "trim": True},
            "NO_CGCE_N2": {"rename": "no_cgce_n2", "cast": "string", "trim": True},
            "NO_CGCE_N2_ING": {"rename": "no_cgce_n2_ing", "cast": "string", "trim": True},
            "NO_CGCE_N2_ESP": {"rename": "no_cgce_n2_esp", "cast": "string", "trim": True},
            "CO_CGCE_N1": {"rename": "co_cgce_n1", "cast": "string", "trim": True},
            "NO_CGCE_N1": {"rename": "no_cgce_n1", "cast": "string", "trim": True},
            "NO_CGCE_N1_ING": {"rename": "no_cgce_n1_ing", "cast": "string", "trim": True},
            "NO_CGCE_N1_ESP": {"rename": "no_cgce_n1_esp", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "CGCE"
    },
    
    {
        "bronze_table": "ncm_cuci",
        "silver_table": "silver_ncm_cuci",
        "key_columns": ["co_cuci_item"],
        "column_mapping": {
            "CO_CUCI_ITEM": {"rename": "co_cuci_item", "cast": "string", "trim": True},
            "NO_CUCI_ITEM": {"rename": "no_cuci_item", "cast": "string", "trim": True},
            "CO_CUCI_SUB": {"rename": "co_cuci_sub", "cast": "string", "trim": True},
            "NO_CUCI_SUB": {"rename": "no_cuci_sub", "cast": "string", "trim": True},
            "CO_CUCI_GRUPO": {"rename": "co_cuci_grupo", "cast": "string", "trim": True},
            "NO_CUCI_GRUPO": {"rename": "no_cuci_grupo", "cast": "string", "trim": True},
            "CO_CUCI_DIVISAO": {"rename": "co_cuci_divisao", "cast": "string", "trim": True},
            "NO_CUCI_DIVISAO": {"rename": "no_cuci_divisao", "cast": "string", "trim": True},
            "CO_CUCI_SEC": {"rename": "co_cuci_sec", "cast": "string", "trim": True},
            "NO_CUCI_SEC": {"rename": "no_cuci_sec", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "CUCI"
    },
    
    {
        "bronze_table": "ncm_isic",
        "silver_table": "silver_ncm_isic",
        "key_columns": ["co_isic_classe"],
        "column_mapping": {
            "CO_ISIC_CLASSE": {"rename": "co_isic_classe", "cast": "string", "trim": True},
            "NO_ISIC_CLASSE": {"rename": "no_isic_classe", "cast": "string", "trim": True},
            "NO_ISIC_CLASSE_ING": {"rename": "no_isic_classe_ing", "cast": "string", "trim": True},
            "NO_ISIC_CLASSE_ESP": {"rename": "no_isic_classe_esp", "cast": "string", "trim": True},
            "CO_ISIC_GRUPO": {"rename": "co_isic_grupo", "cast": "string", "trim": True},
            "NO_ISIC_GRUPO": {"rename": "no_isic_grupo", "cast": "string", "trim": True},
            "NO_ISIC_GRUPO_ING": {"rename": "no_isic_grupo_ing", "cast": "string", "trim": True},
            "NO_ISIC_GRUPO_ESP": {"rename": "no_isic_grupo_esp", "cast": "string", "trim": True},
            "CO_ISIC_DIVISAO": {"rename": "co_isic_divisao", "cast": "string", "trim": True},
            "NO_ISIC_DIVISAO": {"rename": "no_isic_divisao", "cast": "string", "trim": True},
            "NO_ISIC_DIVISAO_ING": {"rename": "no_isic_divisao_ing", "cast": "string", "trim": True},
            "NO_ISIC_DIVISAO_ESP": {"rename": "no_isic_divisao_esp", "cast": "string", "trim": True},
            "CO_ISIC_SECAO": {"rename": "co_isic_secao", "cast": "string", "trim": True},
            "NO_ISIC_SECAO": {"rename": "no_isic_secao", "cast": "string", "trim": True},
            "NO_ISIC_SECAO_ING": {"rename": "no_isic_secao_ing", "cast": "string", "trim": True},
            "NO_ISIC_SECAO_ESP": {"rename": "no_isic_secao_esp", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "ISIC"
    },
    
    {
        "bronze_table": "ncm",
        "silver_table": "silver_ncm",
        "key_columns": ["co_ncm"],
        "column_mapping": {
            "CO_NCM": {"rename": "co_ncm", "cast": "string", "trim": True},
            "NO_NCM_POR": {"rename": "no_ncm_por", "cast": "string", "trim": True},
            "NO_NCM_ESP": {"rename": "no_ncm_esp", "cast": "string", "trim": True},
            "NO_NCM_ING": {"rename": "no_ncm_ing", "cast": "string", "trim": True},
            "CO_UNID": {"rename": "co_unid", "cast": "string", "trim": True},
            "CO_SH6": {"rename": "co_sh6", "cast": "string", "trim": True},
            "CO_PPE": {"rename": "co_ppe", "cast": "string", "trim": True},
            "CO_PPI": {"rename": "co_ppi", "cast": "string", "trim": True},
            "CO_FAT_AGREG": {"rename": "co_fat_agreg", "cast": "string", "trim": True},
            "CO_CUCI_ITEM": {"rename": "co_cuci_item", "cast": "string", "trim": True},
            "CO_CGCE_N3": {"rename": "co_cgce_n3", "cast": "string", "trim": True},
            "CO_SIIT": {"rename": "co_siit", "cast": "string", "trim": True},
            "CO_ISIC_CLASSE": {"rename": "co_isic_classe", "cast": "string", "trim": True},
            "CO_EXP_SUBSET": {"rename": "co_exp_subset", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "NCM"
    },
    
    {
        "bronze_table": "pais_bloco",
        "silver_table": "silver_pais_bloco",
        "key_columns": ["co_pais", "co_bloco"],
        "column_mapping": {
            "CO_PAIS": {"rename": "co_pais", "cast": "string", "trim": True},
            "CO_BLOCO": {"rename": "co_bloco", "cast": "string", "trim": True},
            "NO_BLOCO": {"rename": "no_bloco", "cast": "string", "trim": True},
            "NO_BLOCO_ING": {"rename": "no_bloco_ing", "cast": "string", "trim": True},
            "NO_BLOCO_ESP": {"rename": "no_bloco_esp", "cast": "string", "trim": True}
        },
        "partition_by": None,
        "description": "País-Bloco"
    },
    
    # ========================================
    # FATOS
    # ========================================
    
    {
        "bronze_table": "exp",
        "silver_table": "silver_exp",
        "key_columns": ["ano", "mes", "co_ncm", "co_pais", "sigla_uf_ncm", "co_via", "co_urf"],
        "column_mapping": {
            "CO_ANO": {"rename": "ano", "cast": "int", "trim": False},
            "CO_MES": {"rename": "mes", "cast": "int", "trim": False},
            "CO_NCM": {"rename": "co_ncm", "cast": "string", "trim": True},
            "CO_UNID": {"rename": "co_unid", "cast": "string", "trim": True},
            "CO_PAIS": {"rename": "co_pais", "cast": "string", "trim": True},
            "SG_UF_NCM": {"rename": "sigla_uf_ncm", "cast": "string", "trim": True},
            "CO_VIA": {"rename": "co_via", "cast": "string", "trim": True},
            "CO_URF": {"rename": "co_urf", "cast": "string", "trim": True},
            "QT_ESTAT": {"rename": "qt_estat", "cast": "bigint", "trim": False},
            "KG_LIQUIDO": {"rename": "kg_liquido", "cast": "bigint", "trim": False},
            "VL_FOB": {"rename": "vl_fob", "cast": "bigint", "trim": False}
        },
        "partition_by": ["ano", "mes"],
        "description": "Fato Exportação"
    },
    
    {
        "bronze_table": "imp",
        "silver_table": "silver_imp",
        "key_columns": ["ano", "mes", "co_ncm", "co_pais", "sigla_uf_ncm", "co_via", "co_urf"],
        "column_mapping": {
            "CO_ANO": {"rename": "ano", "cast": "int", "trim": False},
            "CO_MES": {"rename": "mes", "cast": "int", "trim": False},
            "CO_NCM": {"rename": "co_ncm", "cast": "string", "trim": True},
            "CO_UNID": {"rename": "co_unid", "cast": "string", "trim": True},
            "CO_PAIS": {"rename": "co_pais", "cast": "string", "trim": True},
            "SG_UF_NCM": {"rename": "sigla_uf_ncm", "cast": "string", "trim": True},
            "CO_VIA": {"rename": "co_via", "cast": "string", "trim": True},
            "CO_URF": {"rename": "co_urf", "cast": "string", "trim": True},
            "QT_ESTAT": {"rename": "qt_estat", "cast": "bigint", "trim": False},
            "KG_LIQUIDO": {"rename": "kg_liquido", "cast": "bigint", "trim": False},
            "VL_FOB": {"rename": "vl_fob", "cast": "bigint", "trim": False},
            "VL_FRETE": {"rename": "vl_frete", "cast": "bigint", "trim": False},
            "VL_SEGURO": {"rename": "vl_seguro", "cast": "bigint", "trim": False}
        },
        "partition_by": ["ano", "mes"],
        "description": "Fato Importação"
    },
    
    {
        "bronze_table": "exp_mun",
        "silver_table": "silver_exp_mun",
        "key_columns": ["ano", "mes", "co_sh4", "co_pais", "sigla_uf_mun", "co_mun"],
        "column_mapping": {
            "CO_ANO": {"rename": "ano", "cast": "int", "trim": False},
            "CO_MES": {"rename": "mes", "cast": "int", "trim": False},
            "SH4": {"rename": "co_sh4", "cast": "string", "trim": True},
            "CO_PAIS": {"rename": "co_pais", "cast": "string", "trim": True},
            "SG_UF_MUN": {"rename": "sigla_uf_mun", "cast": "string", "trim": True},
            "CO_MUN": {"rename": "co_mun", "cast": "string", "trim": True},
            "KG_LIQUIDO": {"rename": "kg_liquido", "cast": "bigint", "trim": False},
            "VL_FOB": {"rename": "vl_fob", "cast": "bigint", "trim": False}
        },
        "partition_by": ["ano", "mes"],
        "description": "Fato Exp Municipal"
    },
    
    {
        "bronze_table": "imp_mun",
        "silver_table": "silver_imp_mun",
        "key_columns": ["ano", "mes", "co_sh4", "co_pais", "sigla_uf_mun", "co_mun"],
        "column_mapping": {
            "CO_ANO": {"rename": "ano", "cast": "int", "trim": False},
            "CO_MES": {"rename": "mes", "cast": "int", "trim": False},
            "SH4": {"rename": "co_sh4", "cast": "string", "trim": True},
            "CO_PAIS": {"rename": "co_pais", "cast": "string", "trim": True},
            "SG_UF_MUN": {"rename": "sigla_uf_mun", "cast": "string", "trim": True},
            "CO_MUN": {"rename": "co_mun", "cast": "string", "trim": True},
            "KG_LIQUIDO": {"rename": "kg_liquido", "cast": "bigint", "trim": False},
            "VL_FOB": {"rename": "vl_fob", "cast": "bigint", "trim": False}
        },
        "partition_by": ["ano", "mes"],
        "description": "Fato Imp Municipal"
    }
]

print(f"✅ {len(table_config)} tabelas configuradas")

In [0]:
# MAGIC ## 7. Funções de Transformação

# COMMAND ----------

def get_last_processed_timestamp(silver_table_name: str) -> datetime:
    """
    Recupera último timestamp processado da tabela de controle
    
    Args:
        silver_table_name: Nome da tabela Silver
        
    Returns:
        Último timestamp processado ou None se primeira execução
    """
    try:
        row = (
            spark.table("metadata.silver_control_balanca")
            .filter(F.col("table_name") == silver_table_name)
            .select("last_ingestion_timestamp")
            .orderBy(F.col("last_update").desc())
            .limit(1)
            .collect()
        )
        
        if row:
            timestamp = row[0]["last_ingestion_timestamp"]
            print(f"📅 Último processamento: {timestamp}")
            return timestamp
        else:
            print(f"🆕 Primeira execução")
            return None
            
    except Exception as e:
        print(f"⚠️ Controle vazio: {e}")
        return None


def get_bronze_incremental_latest(bronze_table: str, pk_columns: list, last_processed_at: datetime):
    """
    Lê dados incrementais do Bronze via HIVE METASTORE
    
    Args:
        bronze_table: Nome da tabela Bronze (sem database)
        pk_columns: Lista de colunas da chave primária
        last_processed_at: Timestamp do último processamento (watermark)
        
    Returns:
        DataFrame com dados novos (DEDUPLICAÇÃO SERÁ FEITA DEPOIS DA TRANSFORMAÇÃO)
    """
    
    # ✅ Leitura via Hive Metastore
    full_table_name = f"bronze_balancacomercial.{bronze_table}"
    
    try:
        df_bronze = spark.table(full_table_name)
    except Exception as e:
        print(f"❌ Tabela Bronze não existe: {full_table_name}")
        raise
    
    # Filtro incremental
    if last_processed_at:
        df_filtered = df_bronze.filter(F.col("_ingest_ts") > F.lit(last_processed_at))
        records_filtered = df_filtered.count()
        print(f"📊 Registros incrementais: {records_filtered:,}")
    else:
        df_filtered = df_bronze
        records_total = df_filtered.count()
        print(f"📊 Carga FULL: {records_total:,}")
    
    # ✅ RETORNA SEM DEDUPLICAÇÃO
    # A deduplicação será feita DEPOIS da transformação de colunas
    # para evitar conflito entre nomes Bronze (CO_ANO) e Silver (ano)
    
    return df_filtered


def apply_column_mapping(df, column_mapping: dict):
    """
    Aplica transformações de colunas (cast, rename, trim)
    
    Args:
        df: DataFrame Bronze
        column_mapping: Dicionário com regras de transformação
        
    Returns:
        DataFrame transformado
    """
    select_exprs = []
    
    for source_col, rules in column_mapping.items():
        if source_col not in df.columns:
            print(f"⚠️ Coluna {source_col} não encontrada - ignorando")
            continue
        
        # Suporte para mapeamento simples (string)
        if isinstance(rules, str):
            select_exprs.append(F.col(source_col).alias(rules))
            continue
        
        # Mapeamento completo (dict)
        target_col = rules.get("rename", source_col)
        cast_type = rules.get("cast")
        trim_flag = rules.get("trim", False)
        
        col_expr = F.col(source_col)
        
        # Aplicar trim se string
        if trim_flag:
            col_expr = F.trim(col_expr)
        
        # Aplicar cast
        if cast_type:
            col_expr = col_expr.cast(cast_type)
        
        select_exprs.append(col_expr.alias(target_col))
    
    # Preservar metadados técnicos
    technical_cols = ["ingest_date", "_ingest_ts", "_source_path"]
    for col_name in technical_cols:
        if col_name in df.columns:
            select_exprs.append(F.col(col_name))
    
    return df.select(*select_exprs)


def deduplicate_by_pk(df, pk_columns: list):
    """
    Deduplica DataFrame por chave primária (mantém registro mais recente)
    
    Args:
        df: DataFrame já transformado (com nomes Silver)
        pk_columns: Lista de colunas da chave primária (nomes Silver)
        
    Returns:
        DataFrame deduplicado
    """
    
    # Window por PK, ordenado por timestamp descendente
    window_pk = Window.partitionBy(*pk_columns).orderBy(F.col("_ingest_ts").desc())
    
    df_dedup = (
        df
        .withColumn("_rn", F.row_number().over(window_pk))
        .filter(F.col("_rn") == 1)
        .drop("_rn")
    )
    
    records_dedup = df_dedup.count()
    print(f"✅ Registros únicos após deduplicação: {records_dedup:,}")
    
    return df_dedup


def merge_into_silver(df_source, silver_table_name: str, pk_columns: list, partition_by: list):
    """
    Executa MERGE na Silver e registra no Hive Metastore
    
    Args:
        df_source: DataFrame transformado
        silver_table_name: Nome da tabela Silver (ex: 'silver_pais')
        pk_columns: Colunas da chave primária
        partition_by: Colunas para particionamento (ou None)
    """
    
    # Path físico no ADLS Gen2
    silver_path = f"{SILVER_BASE_PATH}/{silver_table_name}"
    full_table_name = f"silver_balancacomercial.{silver_table_name}"
    
    # Verificar se tabela existe
    if not DeltaTable.isDeltaTable(spark, silver_path):
        print(f"🆕 Criando tabela: {full_table_name}")
        
        # Escrever arquivos Delta
        writer = df_source.write.format("delta").mode("overwrite")
        
        if partition_by:
            writer = writer.partitionBy(*partition_by)
            print(f"📂 Particionamento: {partition_by}")
        
        writer.save(silver_path)
        
        # ✅ Registrar no Hive Metastore
        spark.sql(f"""
            CREATE TABLE IF NOT EXISTS {full_table_name}
            USING DELTA
            LOCATION '{silver_path}'
        """)
        
        print(f"✅ Tabela criada e registrada no Hive")
        return
    
    # MERGE incremental (SCD Type 1)
    print(f"🔄 Executando MERGE: {full_table_name}")
    
    delta_silver = DeltaTable.forPath(spark, silver_path)
    merge_condition = " AND ".join([f"silver.{pk} = source.{pk}" for pk in pk_columns])
    
    # Colunas para update (todas exceto PKs)
    update_columns = {col: f"source.{col}" for col in df_source.columns if col not in pk_columns}
    
    # Colunas para insert (todas)
    insert_columns = {col: f"source.{col}" for col in df_source.columns}
    
    (
        delta_silver.alias("silver")
        .merge(df_source.alias("source"), merge_condition)
        .whenMatchedUpdate(set=update_columns)
        .whenNotMatchedInsert(values=insert_columns)
        .execute()
    )
    
    print(f"✅ MERGE concluído")


def update_control_table(silver_table_name: str, max_timestamp: datetime, records_count: int, execution_id: str):
    """
    Atualiza tabela de controle com watermark
    
    Args:
        silver_table_name: Nome da tabela Silver
        max_timestamp: Timestamp máximo processado
        records_count: Quantidade de registros processados
        execution_id: ID da execução
    """
    
    # ✅ CALCULAR timestamp atual ANTES de criar tupla
    current_ts = datetime.now()
    
    control_data = [(silver_table_name, max_timestamp, current_ts, records_count, execution_id)]
    df_new = spark.createDataFrame(
        control_data,
        ["table_name", "last_ingestion_timestamp", "last_update", "records_processed", "execution_id"]
    )
    
    if DeltaTable.isDeltaTable(spark, CONTROL_TABLE_PATH):
        delta_control = DeltaTable.forPath(spark, CONTROL_TABLE_PATH)
        
        (
            delta_control.alias("t")
            .merge(df_new.alias("s"), "t.table_name = s.table_name")
            .whenMatchedUpdate(set={
                "last_ingestion_timestamp": "s.last_ingestion_timestamp",
                "last_update": "s.last_update",
                "records_processed": "t.records_processed + s.records_processed",
                "execution_id": "s.execution_id"
            })
            .whenNotMatchedInsert(values={
                "table_name": "s.table_name",
                "last_ingestion_timestamp": "s.last_ingestion_timestamp",
                "last_update": "s.last_update",
                "records_processed": "s.records_processed",
                "execution_id": "s.execution_id"
            })
            .execute()
        )
        
        print(f"📝 Controle atualizado")
    else:
        df_new.write.format("delta").mode("overwrite").save(CONTROL_TABLE_PATH)
        print(f"🆕 Controle inicializado")

In [0]:
# MAGIC ## 8. Processamento Incremental

# COMMAND ----------

execution_id = datetime.now().strftime("%Y%m%d_%H%M%S")
print(f"🚀 Execution ID: {execution_id}")
print(f"{'='*80}\n")

# Estatísticas da execução
total_tables_processed = 0
total_records_processed = 0
tables_created = []
tables_updated = []
tables_failed = []

for config in table_config:
    
    bronze_table = config["bronze_table"]
    silver_table = config["silver_table"]
    pk_cols = config["key_columns"]
    column_mapping = config["column_mapping"]
    partition_by = config["partition_by"]
    description = config["description"]
    
    print(f"\n{'='*80}")
    print(f"📋 Bronze: bronze_balancacomercial.{bronze_table}")
    print(f"📋 Silver: silver_balancacomercial.{silver_table}")
    print(f"📝 Descrição: {description}")
    print(f"{'='*80}")
    
    try:
        # 1. Verificar se Bronze existe
        try:
            spark.table(f"bronze_balancacomercial.{bronze_table}").limit(1).count()
        except:
            print(f"⚠️ Tabela Bronze não existe: bronze_balancacomercial.{bronze_table}")
            tables_failed.append(silver_table)
            continue
        
        # 2. Recuperar último timestamp processado
        last_processed_at = get_last_processed_timestamp(silver_table)
        
        # 3. Ler dados incrementais do Bronze (SEM deduplicação ainda)
        df_latest = get_bronze_incremental_latest(bronze_table, pk_cols, last_processed_at)
        
        # 4. Verificar se há dados novos
        if df_latest.count() == 0:
            print(f"ℹ️ Sem dados novos para processar")
            continue
        
        # 5. Aplicar transformações (rename, cast, trim)
        df_transformed = apply_column_mapping(df_latest, column_mapping)
        
        # 6. ✅ DEDUPLICAR APÓS TRANSFORMAÇÃO (com nomes Silver corretos)
        df_transformed = deduplicate_by_pk(df_transformed, pk_cols)
        
        # 7. Verificar se é tabela nova
        is_new_table = not spark.catalog.tableExists(f"silver_balancacomercial.{silver_table}")
        
        # 8. MERGE na Silver
        merge_into_silver(df_transformed, silver_table, pk_cols, partition_by)
        
        # 9. Atualizar controle
        max_timestamp = df_transformed.agg(F.max("_ingest_ts")).collect()[0][0]
        records_count = df_transformed.count()
        
        update_control_table(silver_table, max_timestamp, records_count, execution_id)
        
        # 10. Registrar estatísticas
        total_tables_processed += 1
        total_records_processed += records_count
        
        if is_new_table:
            tables_created.append(silver_table)
        else:
            tables_updated.append(silver_table)
        
        print(f"✅ {silver_table} processado com sucesso!")
        
    except Exception as e:
        print(f"❌ ERRO ao processar {silver_table}")
        print(f"   Erro: {str(e)}")
        tables_failed.append(silver_table)
        
        import traceback
        traceback.print_exc()

print(f"\n{'='*80}")
print(f"🎉 PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")

In [0]:
## 9. Relatório de Execução

# COMMAND ----------

print(f"""
╔══════════════════════════════════════════════════════════════════╗
║            RELATÓRIO - SILVER BALANÇA COMERCIAL                  ║
╚══════════════════════════════════════════════════════════════════╝

📊 ESTATÍSTICAS GERAIS:
   • Execution ID: {execution_id}
   • Tabelas processadas: {total_tables_processed}
   • Registros processados: {total_records_processed:,}
   • Protocolo: abfss:// (ADLS Gen2)

✅ TABELAS CRIADAS ({len(tables_created)}):
{chr(10).join(f'   • {t}' for t in tables_created) if tables_created else '   (nenhuma)'}

🔄 TABELAS ATUALIZADAS ({len(tables_updated)}):
{chr(10).join(f'   • {t}' for t in tables_updated) if tables_updated else '   (nenhuma)'}

{f'❌ TABELAS COM FALHA ({len(tables_failed)}):' if tables_failed else ''}
{chr(10).join(f'   • {t}' for t in tables_failed) if tables_failed else ''}

🗄️ ARQUITETURA:
   • Bronze (leitura): Hive Metastore → bronze_balancacomercial.*
   • Silver (escrita): ADLS Gen2 + Hive → silver_balancacomercial.*
   • Controle: metadata.silver_control_balanca

📂 LOCALIZAÇÃO FÍSICA:
   • Silver Tables: {SILVER_BASE_PATH}
   • Control Table: {CONTROL_TABLE_PATH}

🔗 CONEXÃO POWER BI:
   • Catalog: hive_metastore
   • Schema: silver_balancacomercial
   • Autenticação: Configurada no cluster (Service Principal/Managed Identity)

⏱️ PRÓXIMA EXECUÇÃO:
   • Apenas dados com _ingestion_timestamp > último processamento
   • Processamento incremental automático
""")